## Randomized algorithm for 2-SAT

<br>
University of Miami
<br>
REU summer 2022
<br>
Burton Rosenberg.
<br>
<br>last update: May 30, 2022

A boolean formula is a formula over the logical arithmetic of true and false values, 
with operations AND, OR and NOT.

A solution to a boolean formula is a setting of the variables in the formula to 
true or false so that the computed value of the formula is true. 
The values assigned is a called a _satisfying assignment_.

A boolean formula can be put into _Conjunctive Normal Form_ (CNF). Such of form is the AND of clauses, each clause is the OR of variables or their complements, for instance, 

$$
D = (a \lor b \lor \neg c \lor e)\land (c \lor \neg b \lor a) \land (b) 
$$

The clauses in the formula are, 

\begin{eqnarray*}
A &=& a \lor b \lor \neg c \lor e \\
B &=& c \lor \neg b \lor a \\
C &=& b \\
\end{eqnarray*}

We shall represent a SAT instance as a list of clauses, where each clause is a list of pairs, where each pair is a variable name and the integer 0 if the variable appears uncomplement, and 1 if the variable appears complemented.

A truth assignment is a dictionary from variable names to the boolean value True or False. The problem of SAT is to find a truth assignment so that the CNF evaluates to true. Such an assignment is called a satisfying assignment. 

If each clause has exactly 2 variables, the CNF is called an instance of 2-SAT ("two satisfiability"). We explore this method of solving 2-SAT:

- Start with any truth assignment.
- Repeat r times,
  - If the truth assignment satisfies the 2-SAT, halt.
  - Else pick any false clause, pick one of the two variables in the clause, and negate its value.


A bit of math shows that for $n$ variables, with probably 1/2 this will find a satisfying assignment, if one exists, after $2\,n^2$ variable flips. Hence repeating $2m\,\,n^2$ will find a satisfyying assignment, if one exists, with probabilty $1/2^m$.

This method does not work for any k-SAT, with k three or larger.



In [1]:
import random

class SAT_Instance:
    
    def __init__(self, cnf_formula):
        self.cnf = cnf_formula
        self.next_i = 0
        self.t_a = self.create_t_a(cnf_formula,False)
        
    def create_t_a(self,cnf,t):
        t_a = {}
        for clause in cnf:
            for var in clause:
                t_a[var[0]] = t
        return t_a
    
    def is_2_sat(self):
        for clause in self.cnf:
            if len(clause)!=2:
                return False
        return True

    def is_clause_satisfied(self,clause,assign):
        t = False
        for v in clause:
            t1 = assign[v[0]]
            if v[1]==1:
                t1 = not t1
            t = t or t1
        return t
        
    def is_satisfied(self,assign):
        for clause in self.cnf:
            t = self.is_clause_satisfied(clause,assign)
            if not t:
                return False
        return True
        
    def change_one(self):
        l = [i for i in range(self.next_i,len(self.cnf))]
        l += [i for i in range(0,self.next_i)]
        for j in l:
            if not self.is_clause_satisfied(self.cnf[j],self.t_a):
                b = random.randint(0,1)
                self.t_a[self.cnf[j][b][0]] = not self.t_a[self.cnf[j][b][0]]
                self.next_i = (j+1)%len(self.cnf)
                return False
        return True
    
    def random_walk(self,m,init=None,verbose=False):
        assert self.is_2_sat()
        if init==None:
            self.t_a = self.create_t_a(self.cnf,False)
        else:
            self.t_a in init
        self.next_i = 0
        j = 0
        if verbose:
            print(j,self.t_a)
        while not self.change_one():
            if verbose:
                print(j,self.t_a)
            j += 1
            if j>(2*m*len(self.t_a)**2):
                return False
        return True
        
    def __repr__(self):

        def p_aux(s, d):
            f = False
            s += '('
            for v in d:
                if f:
                    s += ' OR '
                else:
                    f = True
                if v[1]==0:
                    s += f'{v[0]}'
                else:
                    s += f'~{v[0]}'
            s += f')'
            return s
        
        f = False
        s = ''
        for d in self.cnf:
            if f:
                s += ' AND '
            else:
                f = True
            s = p_aux(s, d)
        return s


In [2]:

# examples

eqn_7_2 = [[('x1',0),('x2',1)],
           [('x1',1),('x3',1)],
           [('x1',0),('x2',0)],
           [('x4',0),('x3',1)],
           [('x4',0),('x1',1)]]

sat_7_2 = SAT_Instance(eqn_7_2)
print(sat_7_2)

print(sat_7_2.is_satisfied({'x1':True, 'x3': False, 'x4':True, 'x2':False,}))

E = [[('a',1),('b',1)],
     [('b',0),('c',0)]]

e_sat =(SAT_Instance(E))
print(e_sat)
print(e_sat.is_satisfied({'b':False, 'a':True, 'c':True, 'e':False }))




E = [[('a',1),('b',1)],[('b',0),('c',0)]]
e_sat =(SAT_Instance(E))
assign = {'b':False, 'a':True, 'c':True, 'e':False }

e_sat.is_satisfied(assign)
e_sat.is_2_sat()
e_sat.random_walk(3,verbose=True)


(x1 OR ~x2) AND (~x1 OR ~x3) AND (x1 OR x2) AND (x4 OR ~x3) AND (x4 OR ~x1)
True
(~a OR ~b) AND (b OR c)
True
0 {'a': False, 'b': False, 'c': False}
0 {'a': False, 'b': False, 'c': True}


True

In [3]:

# examples

ex_7_5 = [[('x',0),('y',0)],
          [('x',0),('y',1)],
          [('x',1),('y',0)],
          [('x',1),('y',1)]]

print(SAT_Instance(ex_7_5))


fig_7_33 = [[('x1',0),('x1',0),('x2',0),],
            [('x1',1),('x2',1),('x2',1),],
            [('x1',1),('x2',0),('x2',0),],]

sat_7_33 = SAT_Instance(fig_7_33)
print(sat_7_33,)
print(sat_7_33.t_a)

A = [('a',0),('b',0),('c',1),('e',0)]
B = [('c',0),('b',1),('a',0)]
C = [('b',0)]

D = [A,B,C]
print(SAT_Instance(D))

eqn_7_2 = [[('x1',0),('x2',1)],[('x1',1),('x3',1)],[('x1',0),('x2',0)],[('x4',0),('x3',1)],[('x4',0),('x1',1)]]
sat_7_2 = SAT_Instance(eqn_7_2)
print(sat_7_2)

(x OR y) AND (x OR ~y) AND (~x OR y) AND (~x OR ~y)
(x1 OR x1 OR x2) AND (~x1 OR ~x2 OR ~x2) AND (~x1 OR x2 OR x2)
{'x1': False, 'x2': False}
(a OR b OR ~c OR e) AND (c OR ~b OR a) AND (b)
(x1 OR ~x2) AND (~x1 OR ~x3) AND (x1 OR x2) AND (x4 OR ~x3) AND (x4 OR ~x1)


In [4]:
sat_7_2.random_walk(3,verbose=True)

0 {'x1': False, 'x2': False, 'x3': False, 'x4': False}
0 {'x1': True, 'x2': False, 'x3': False, 'x4': False}
1 {'x1': False, 'x2': False, 'x3': False, 'x4': False}
2 {'x1': True, 'x2': False, 'x3': False, 'x4': False}
3 {'x1': True, 'x2': False, 'x3': False, 'x4': True}


True

In [5]:
sat_7_2.is_satisfied(sat_7_2.t_a)

True

In [6]:
r = [[('x1',0),('x2',1)],[('x1',1),('x3',1)],[('x1',0),('x2',0)],[('x4',0),('x3',1)],[('x4',0),('x1',1)],
[('x5',0),('x6',1)],[('x1',0),('x4',1)],[('x7',0),('x3',0)],[('x2',0),('x3',1)],[('x5',0),('x4',1)]]

In [7]:
r_cnf = SAT_Instance(r)
print(r_cnf)
r_cnf.random_walk(3,verbose=True)

(x1 OR ~x2) AND (~x1 OR ~x3) AND (x1 OR x2) AND (x4 OR ~x3) AND (x4 OR ~x1) AND (x5 OR ~x6) AND (x1 OR ~x4) AND (x7 OR x3) AND (x2 OR ~x3) AND (x5 OR ~x4)
0 {'x1': False, 'x2': False, 'x3': False, 'x4': False, 'x5': False, 'x6': False, 'x7': False}
0 {'x1': False, 'x2': True, 'x3': False, 'x4': False, 'x5': False, 'x6': False, 'x7': False}
1 {'x1': False, 'x2': True, 'x3': True, 'x4': False, 'x5': False, 'x6': False, 'x7': False}
2 {'x1': False, 'x2': False, 'x3': True, 'x4': False, 'x5': False, 'x6': False, 'x7': False}
3 {'x1': True, 'x2': False, 'x3': True, 'x4': False, 'x5': False, 'x6': False, 'x7': False}
4 {'x1': True, 'x2': False, 'x3': True, 'x4': True, 'x5': False, 'x6': False, 'x7': False}
5 {'x1': True, 'x2': True, 'x3': True, 'x4': True, 'x5': False, 'x6': False, 'x7': False}
6 {'x1': True, 'x2': True, 'x3': True, 'x4': False, 'x5': False, 'x6': False, 'x7': False}
7 {'x1': True, 'x2': True, 'x3': False, 'x4': False, 'x5': False, 'x6': False, 'x7': False}
8 {'x1': True, 'x

True

In [38]:
three_three = [[('a1',0),('b1',0)],[('b1',1),('c1',0)],
               [('a2',1),('b2',0)],[('b2',0),('c2',0)],[('c2',1),('a2',0)],
               [('a3',1),('b3',0)],[('b3',1),('c3',0)],[('c3',1),('a3',1)],
               [('a1',1),('b2',1)],[('c2',1),('a3',0)],[('c3',0),('a1',0)]
              ]

cnf33 = SAT_Instance(three_three)
print(cnf33)

(a1 OR b1) AND (~b1 OR c1) AND (~a2 OR b2) AND (b2 OR c2) AND (~c2 OR a2) AND (~a3 OR b3) AND (~b3 OR c3) AND (~c3 OR ~a3) AND (~a1 OR ~b2) AND (~c2 OR a3) AND (c3 OR a1)


In [39]:
cnf33.random_walk(3)

True

In [40]:
cnf33.t_a


{'a1': False,
 'b1': True,
 'c1': True,
 'a2': False,
 'b2': True,
 'c2': False,
 'a3': False,
 'b3': False,
 'c3': True}